In [8]:
import boto3
# print(f"SageMaker Version: {sagemaker.__version__}")
# print(f"Installation Path: {sagemaker.__file__}")
import sagemaker
from sagemaker.pytorch import PyTorch

from botocore.exceptions import ClientError

boto_session = boto3.Session(region_name='us-west-1')

# Get the caller identity
sts_client = boto3.client('sts')
identity = sts_client.get_caller_identity()
arn = identity['Arn']
username = arn.split('/')[-1]


print(f"Account: {identity['Account']}")
print(f"UserId: {identity['UserId']}")
print(f"ARN: {identity['Arn']}")
print(f"Your AWS Username is: {username}")


Account: 253490779227
UserId: AIDATWBJ2KRN6C42QYYWF
ARN: arn:aws:iam::253490779227:user/amannindra2
Your AWS Username is: amannindra2


In [14]:
# Stop a specific job
sm_client = boto3.client('sagemaker', region_name='us-west-1')
sm_client.stop_training_job(TrainingJobName='pytorch-training-2025-11-25-02-50-20-903')

{'ResponseMetadata': {'RequestId': '594fa00f-0c0a-4a56-b537-3e502dbe0eef',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '594fa00f-0c0a-4a56-b537-3e502dbe0eef',
   'strict-transport-security': 'max-age=47304000; includeSubDomains',
   'x-frame-options': 'DENY',
   'content-security-policy': "frame-ancestors 'none'",
   'cache-control': 'no-cache, no-store, must-revalidate',
   'x-content-type-options': 'nosniff',
   'content-type': 'application/x-amz-json-1.1',
   'date': 'Tue, 25 Nov 2025 03:12:26 GMT',
   'content-length': '0'},
  'RetryAttempts': 0}}

In [12]:

# REPLACE THIS STRING with the actual Role ARN you created in IAM
role = "arn:aws:iam::253490779227:role/service-role/AmazonSageMakerAdminIAMExecutionRole" 


sagemaker_session = sagemaker.Session(boto_session=boto_session)

print(f"SageMaker Session Region: {sagemaker_session.boto_region_name}")
print(f"Default Bucket: {sagemaker_session.default_bucket()}")


SageMaker Session Region: us-west-1
Default Bucket: sagemaker-us-west-1-253490779227


In [15]:
s3_input_data = 's3://animal-classification-dss/' 

# Configure boto3 with retry logic for large uploads
from botocore.config import Config

boto_config = Config(
    retries={
        'max_attempts': 10,
        'mode': 'adaptive'
    },
    connect_timeout=60,
    read_timeout=120
)

# Recreate session with retry config
boto_session = boto3.Session(region_name='us-west-1')
sagemaker_session = sagemaker.Session(
    boto_session=boto_session,
    default_bucket='sagemaker-us-west-1-253490779227'
)

estimator = PyTorch(
    entry_point='train.py',          # The script we created above
    source_dir='.',                  # Uploads the current directory (with train.py) to AWS
    # dependencies=['requirements_sagemaker.txt'],  # Use minimal requirements
    role=role,                       # Your IAM Role ARN
    framework_version='2.1',         # PyTorch version
    py_version='py310',              # Python version
    instance_count=1,
    instance_type='ml.g4dn.xlarge',     # Use 'ml.g4dn.xlarge' for GPU training
    hyperparameters={
        'epochs': 5,
        'batch-size': 32
    },
    sagemaker_session=sagemaker_session
)
estimator.fit({'training': s3_input_data})


INFO:botocore.credentials:Found credentials in shared credentials file: ~/.aws/credentials
INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: pytorch-training-2025-11-25-03-12-35-197


2025-11-25 03:16:01 Starting - Starting the training job
2025-11-25 03:16:01 Pending - Training job waiting for capacity...
2025-11-25 03:16:15 Pending - Preparing the instances for training...
2025-11-25 03:16:41 Downloading - Downloading input data...
2025-11-25 03:17:21 Downloading - Downloading the training image........................
2025-11-25 03:21:18 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.10/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.10/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be

In [9]:


# Initialize S3 client
s3_client = boto3.client('s3', region_name='us-west-1')

# Or initialize S3 resource (higher-level interface)
s3_resource = boto3.resource('s3', region_name='us-west-1')

# Your bucket name
bucket_name = 'animal-classification-dss'

# Example 1: List objects in the bucket
# try:
#     response = s3_client.list_objects_v2(Bucket=bucket_name)
#     if 'Contents' in response:
#         print("Objects in bucket:")
#         for obj in response['Contents']:
#             print(f"  {obj['Key']} - Size: {obj['Size']} bytes")
#     else:
#         print("Bucket is empty")
# except ClientError as e:
#     print(f"Error accessing bucket: {e}")

# # Example 2: Upload a file
# def upload_file(file_path, object_key):
#     try:
#         s3_client.upload_file(file_path, bucket_name, object_key)
#         print(f"File {file_path} uploaded as {object_key}")
#     except ClientError as e:
#         print(f"Error uploading file: {e}")

# # Example 3: Download a file
# def download_file(object_key, local_file_path):
#     try:
#         s3_client.download_file(bucket_name, object_key, local_file_path)
#         print(f"File {object_key} downloaded to {local_file_path}")
#     except ClientError as e:
#         print(f"Error downloading file: {e}")

# Example 4: Using S3 resource (alternative approach)
bucket = s3_resource.Bucket(bucket_name)

print(bucket)

# List objects using resource


# for obj in bucket.objects.all():
#     print(f"Object: {obj.key}")

# Upload using resource
# bucket.upload_file('local_file.txt', 'remote_file.txt')

# Download using resource
# bucket.download_file('remote_file.txt', 'local_file.txt')


s3.Bucket(name='animal-classification-dss')
